In [52]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import warnings
import geocoder

ModuleNotFoundError: No module named 'geocoder'

# Part 1

In [3]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

In [11]:
table = soup.find('table')
print(table)

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [15]:
trows = table.find_all('tr')

In [21]:
rows=[]
for tr in trows:
    i = tr.find_all('td')
    if i:
        rows.append(i)
list=[]
for row in rows:
    Postcode = row[0].text.rstrip()
    Borough = row[1].text.rstrip()
    Neighbourhood = row[2].text.rstrip()
    if Borough != 'Not assigned':
        if Neighbourhood != 'Not assigned':
            list.append([Postcode,Borough,Neighbourhood])

In [29]:
cols = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(list, columns=cols)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [32]:
df.to_csv(r'C:\Users\Agha Abdul Rauf\Desktop\Data Science Journey\Projects\Recommendation System Using Clustering' + 'PostalCodes.csv', index=False)

# Part 2

In [49]:
df_cords = pd.read_csv('Geospatial_Coordinates.csv')

In [50]:
df_cords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [59]:
df['Lat'] = np.nan
df['Long'] = np.nan
for i in df.index:
    cord = df_cords['Postal Code'] == df.loc[i, 'Postcode']
    df.at[i,'Lat'] = df_cords.loc[cord,'Latitude'].values
    df.at[i,'Long'] = df_cords.loc[cord,'Longitude'].values
df.head()

,Postcode,Borough,Neighbourhood,Lat,Long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [61]:
#Groupby Postal Code and Join for Single Data entity
df = df.groupby('Postcode').agg({'Borough':'first', 'Neighbourhood': ','.join}).reset_index()

In [62]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Part 3